## **Combining Data for use in Tableau**

We'd like to create an interactive dashboard for tableau, but to do that, we'll need to combine our data and add some new columns. Some features we'll need to add to the data include:
* `date_announcement_normalized` - an ordinal column for the date that can be used to align states based on their closing/reopening announcement date. Thinking of using a +/- shutdown date to create this column.
* `is_reopen` - a boolean column that will say whether the row is from the state's opening or closing phase. Will do this by applying a True/False to all reopening or closing dataframes before doing a union.

In [22]:
import os
import csv
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone

In [30]:
mi_reopen = pd.read_csv('./data/sentiment_data/MI_reopen_sentiment.csv')
oh_reopen = pd.read_csv('./data/sentiment_data/OH_reopen_sentiment.csv')
mi_shutdown = pd.read_csv('./data/sentiment_data/MI_shutdown_sentiment.csv')
oh_shutdown = pd.read_csv('./data/sentiment_data/OH_shutdown_sentiment.csv')
tx_reopen = pd.read_csv('./data/sentiment_data/TX_reopen_sentiment.csv')
ga_reopen = pd.read_csv('./data/sentiment_data/GA_reopen_sentiment.csv')
il_reopen = pd.read_csv('./data/sentiment_data/IL_reopen_sentiment.csv')
tx_shutdown = pd.read_csv('./data/sentiment_data/TX_shutdown_sentiment.csv')
ga_shutdown = pd.read_csv('./data/sentiment_data/GA_shutdown_sentiment.csv')
il_shutdown = pd.read_csv('./data/sentiment_data/IL_shutdown_sentiment.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3,4,5,6,8,12,13,18,19,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
cities_to_remove = ['eagle river ']

In [46]:
df = pd.read_csv(f'./data/sentiment_data//MI_shutdown_sentiment.csv')
df.drop_duplicates(subset=['username', 'date'], keep='first', inplace=True) # need to remove any potential duplicates from overlapping city areas

# Clean up final df
df = df[(df['username'] != 'username') & # Removes headers leftover from scraping
       (~df['city'].isin(cities_to_remove))] # Removes the cities that we need to remove
df.dropna(subset=['text', 'date'], inplace=True) # There were some nulls in the text and date column that are likely the result of deleted/private tweets
df.to_csv(f'./data/sentiment_data//MI_shutdown_sentiment2.csv', index=False)

In [35]:
# mi_reopen.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/MI_reopen_sentiment.csv', index=False)
# oh_reopen.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/OH_reopen_sentiment.csv', index=False)
# mi_shutdown.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/MI_shutdown_sentiment.csv', index=False)
# oh_shutdown.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/OH_shutdown_sentiment.csv', index=False)
# tx_reopen.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/TX_reopen_sentiment.csv', index=False)
# il_reopen.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/IL_reopen_sentiment.csv', index=False)
# ga_reopen.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/GA_reopen_sentiment.csv', index=False)
# tx_shutdown.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/TX_shutdown_sentiment.csv', index=False)
# il_shutdown.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/IL_shutdown_sentiment.csv', index=False)
ga_shutdown.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/GA_shutdown_sentiment.csv', index=False)

In [27]:
def rename_date_col(rel_path='./data/sentiment_data'):
    filenames = [filename for filename in os.listdir(rel_path) if filename.endswith('.csv')]
    for filename in filenames:
        data = pd.read_csv(f'./data/sentiment_data/{filename}')
        data.rename(columns={'formatted_date' : 'date'}).to_csv('./data/sentiment_data/MI_reopen_sentiment.csv', index=False)

In [31]:
rename_date_col()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (3,4,5,6,8,12,13,18,19,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [51]:
def combine_and_add_cols(rel_path='./data/sentiment_data'):
    '''
    This function combines and adds columns to the csvs that
    contain sentiment analysis data. Specifically, this function
    adds the following features:
    
    date_ord: creates an ordinal value from the date column
    
    -----------------------
    Parameters:
    
    rel_path : str, the path where all of your csvs are stored
    '''    
    # Getting all of a state's filenames
    filenames = [filename for filename in os.listdir(rel_path) if filename.endswith('.csv')]
    
    # Create new df so the final df doesn't keep appending if you need to run again
    # First, we'll need to get the headers - thanks Tyler on SO for this
    # https://stackoverflow.com/questions/24962908/how-can-i-read-only-the-header-column-of-a-csv-file-using-python
#     with open(f'{rel_path}/{filenames[0]}', 'r') as f:
#         reader = csv.DictReader(f)
#         fieldnames = reader.fieldnames

#     # Write a clean df
#     pd.DataFrame(columns=fieldnames + ['']).to_csv(f'{rel_path}/combined/all_states_and_dates_sentiment.csv', index=False) # THIS DIDNT" NEED TO BE A VARIABLE ++++ REMOVE WHEN DONE
    
    # We'll need this dict for later, the values are organized as [shutdown_date_announced, reopening_date]
    timeline_dict = {
        'FL': ['2020-04-01', '2020-04-29'],
        'TX': ['2020-03-31', '2020-05-18'],
        'NJ': ['2020-03-16', '2020-06-09'],
        'NY': ['2020-03-22', '2020-06-13'],
        'IL': ['2020-03-20', '2020-05-29'],
        'GA': ['2020-04-02', '2020-06-01'],
        'OH': ['2020-03-22', '2020-05-12'],
        'MI': ['2020-03-23', '2020-05-18']
    }
        
    # Main combination loop
    for filename in filenames:
        df = pd.read_csv(f'{rel_path}/{filename}')
        
        #-----------IS REOPEN-------------
        '''
        We want to add a column that tells us if the row is 
        from reopening or shutdown data, we'll do this by
        referencing the filenames since that data is included there
        '''
        
        if filename.split('_')[1] == 'shutdown':
            df['is_reopen'] = False
            is_reopen = False # Setting this for later use
        else:
            df['is_reopen'] = True
            is_reopen = True # Setting this for later use
            
        
        #--------ORDINAL NORMALIZATION-----------
        '''
        The following code will set the shutdown/reopening 
        date as 0, so we can align states on a graph based on this date.
        '''
        
        # converting date to datetime
        df['date'] = pd.to_datetime(df['date'])

        # setting date as index
        df.set_index('date', inplace=True, drop=False)
        
        # State name pulled from filename
        state_name = filename.split('_')[0]
        
        announce_date = timeline_dict[state_name][is_reopen]
        
        # Setting the announcement day and hour (since not every dataset has noon, 
        # we're using the last tweet from the announcement day as our midpoint
        announcement_day_hour = df[f'{announce_date}']['date'][len(df[announce_date])//2] # this will pick the middle entry from the announcement day

        # Take the time delta in seconds divided by 60 ** 2 to get nubmer of hours
        df['date_announcement_normalized'] = df.apply(lambda x: (x['date'] - announcement_day_hour).total_seconds() // 60 ** 2, axis=1)

            
        #----------WRITE TO FINAL DF--------
        
        if filename == filenames[0]:
            # Write to final df
            df.to_csv(f'{rel_path}/combined/all_states_and_dates_sentiment.csv', index=False)
        else:
            # Append to final df
            df.to_csv(f'{rel_path}/combined/all_states_and_dates_sentiment.csv', index=False, mode='a', header=False)

In [52]:
combine_and_add_cols('./data/sentiment_data')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (3,4,5,6,8,12,13,18,19,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


ParserError: Unknown string format: formatted_date